#### New QAT frontdoor getting started (EXPERIMENTAL)

This is a demo of the new QAT 'frontdoor' which uses the experimental QAT pipelines.

We aren't using this in production yet and your mileage may vary.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from qat import QAT

In [ ]:
prog = """
OPENQASM 3;
bit[2] c;
qubit[2] q;
h q;
measure q -> c;
"""

In [ ]:
qat = QAT("../qatconfig.eg.yaml")
qat.set_default_pipeline("echo")

In [ ]:
inst, metrics = qat.compile(prog)
res, metrics = qat.execute(inst)
res

In [ ]:
qat.set_default_pipeline("rtcs")
inst, metrics = qat.compile(prog)
res, metrics = qat.execute(inst)
res

In [ ]:
inst, metrics = qat.compile(prog, pipeline="echo")
res, metrics = qat.execute(inst, pipeline="echo")
res

In [ ]:
from qat.pipelines import DefaultCompile, DefaultExecute, DefaultPostProcessing
from qat.purr.backends.echo import get_default_echo_hardware

echo16 = get_default_echo_hardware(qubit_count=16)

In [ ]:
qat.add_pipeline(
    "echo16",
    compile_pipeline=DefaultCompile(echo16),
    execute_pipeline=DefaultExecute(echo16),
    postprocess_pipeline=DefaultPostProcessing(echo16),
    engine=echo16.create_engine(),
)

In [ ]:
inst, metrics = qat.compile(prog, pipeline="echo16")
res, metrics = qat.execute(inst, pipeline="echo16")
res